In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import nltk
from nltk.tokenize import word_tokenize
import gensim.downloader as api

# Загрузка данных (пример с IMDB)
# На реальных данных используйте:
# data = pd.read_csv('ваш_файл.csv')
data = pd.DataFrame({
    'text': [
        'Этот фильм просто потрясающий, я в восторге!',
        'Ужасный фильм, худший из того, что я видел',
        'Отличный сюжет и великолепная игра актеров',
        'Не рекомендую, пустая трата времени',
        'Шедевр кинематографа, обязательно к просмотру'
    ],
    'label': [1, 0, 1, 0, 1]  # 1 - положительный, 0 - отрицательный
})

# Загрузка предобученной модели Word2Vec
word2vec_model = api.load('word2vec-google-news-300')

# Функция для получения эмбеддинга текста
def get_document_embedding(text, model):
    # Токенизация
    tokens = word_tokenize(text.lower())
    
    # Получение эмбеддингов для каждого слова, если оно есть в модели
    embeddings = [model[word] for word in tokens if word in model]
    
    # Если ни одного слова не найдено, возвращаем нулевой вектор
    if len(embeddings) == 0:
        return np.zeros(model.vector_size)
    
    # Вычисление среднего вектора для всего документа
    return np.mean(embeddings, axis=0)

# Преобразование текстов в эмбеддинги
X = np.array([get_document_embedding(text, word2vec_model) for text in data['text']])
y = data['label']

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Обучение модели логистической регрессии
model = LogisticRegression()
model.fit(X_train, y_train)

# Прогнозирование и оценка
y_pred = model.predict(X_test)
print(f"Точность: {accuracy_score(y_test, y_pred)}")
print(classification_report(y_test, y_pred))

# Проверка модели на новом тексте
new_text = "Этот фильм действительно впечатляет своей глубиной"
new_embedding = get_document_embedding(new_text, word2vec_model)
prediction = model.predict([new_embedding])[0]
print(f"Предсказание для '{new_text}': {'положительный' if prediction == 1 else 'отрицательный'}")

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from transformers import BertTokenizer, BertModel
import torch

# Загрузка тех же данных
data = pd.DataFrame({
    'text': [
        'Этот фильм просто потрясающий, я в восторге!',
        'Ужасный фильм, худший из того, что я видел',
        'Отличный сюжет и великолепная игра актеров',
        'Не рекомендую, пустая трата времени',
        'Шедевр кинематографа, обязательно к просмотру'
    ],
    'label': [1, 0, 1, 0, 1]  # 1 - положительный, 0 - отрицательный
})

# Загрузка токенизатора и модели BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained('bert-base-multilingual-cased')

# Функция для получения эмбеддинга с использованием BERT
def get_bert_embedding(text, tokenizer, model):
    # Токенизация и подготовка входных данных
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    
    # Получение эмбеддингов из BERT
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Используем векторное представление [CLS] токена в качестве эмбеддинга всего предложения
    return outputs.last_hidden_state[:, 0, :].numpy().flatten()

# Преобразование текстов в эмбеддинги BERT
X = np.array([get_bert_embedding(text, tokenizer, model) for text in data['text']])
y = data['label']

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Обучение модели случайного леса
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Прогнозирование и оценка
y_pred = rf_model.predict(X_test)
print(f"Точность: {accuracy_score(y_test, y_pred)}")
print(classification_report(y_test, y_pred))

# Проверка модели на новом тексте
new_text = "Этот фильм действительно впечатляет своей глубиной"
new_embedding = get_bert_embedding(new_text, tokenizer, model)
prediction = rf_model.predict([new_embedding])[0]
print(f"Предсказание для '{new_text}': {'положительный' if prediction == 1 else 'отрицательный'}")

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import gensim.downloader as api
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Загрузка данных (пример)
data = pd.DataFrame({
    'text': [
        'Этот фильм просто потрясающий, я в восторге!',
        'Ужасный фильм, худший из того, что я видел',
        'Отличный сюжет и великолепная игра актеров',
        'Не рекомендую, пустая трата времени',
        'Шедевр кинематографа, обязательно к просмотру',
        'Сценарий слабый, но актеры старались',
        'В целом неплохо, но ожидал большего',
        'Разочарование года, не советую тратить деньги',
        'Потрясающий визуальный ряд и спецэффекты',
        'Затянуто и скучно, уснул в середине фильма'
    ],
    'label': [1, 0, 1, 0, 1, 0, 0, 0, 1, 0]  # 1 - положительный, 0 - отрицательный
})

# Загрузка стоп-слов и инициализация лемматизатора
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
stop_words = set(stopwords.words('russian') + stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# Загрузка предобученной модели GloVe
glove_model = api.load('glove-wiki-gigaword-300')

# Функция предобработки текста
def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token.isalpha() and token not in stop_words]
    return tokens

# Функция для получения эмбеддинга документа
def get_glove_embedding(text, model):
    tokens = preprocess_text(text)
    embeddings = [model[word] for word in tokens if word in model]
    if len(embeddings) == 0:
        return np.zeros(model.vector_size)
    return np.mean(embeddings, axis=0)

# Преобразование текстов в эмбеддинги
X = np.array([get_glove_embedding(text, glove_model) for text in data['text']])
y = data['label']

# Создание пайплайна с масштабированием и SVM
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('svm', SVC(probability=True))
])

# Определение параметров для поиска
param_grid = {
    'svm__C': [0.1, 1, 10, 100],
    'svm__kernel': ['linear', 'rbf'],
    'svm__gamma': ['scale', 'auto', 0.1, 0.01]
}

# Поиск лучших гиперпараметров с кросс-валидацией
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='f1')
grid_search.fit(X, y)

print(f"Лучшие параметры: {grid_search.best_params_}")
print(f"Лучший f1-score: {grid_search.best_score_:.4f}")

# Оценка лучшей модели с использованием кросс-валидации
best_model = grid_search.best_estimator_
cv_scores = cross_val_score(best_model, X, y, cv=5, scoring='f1')
print(f"Средний f1-score при 5-кратной кросс-валидации: {cv_scores.mean():.4f}")

# Обучение и предсказание для визуализации матрицы неточностей
best_model.fit(X, y)
y_pred = best_model.predict(X)

# Визуализация матрицы неточностей
cm = confusion_matrix(y, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Негативный', 'Позитивный'])
disp.plot(cmap='Blues')
plt.title('Матрица неточностей')
plt.show()

# Анализ ошибок классификации
misclassified = data[y != y_pred].copy()
misclassified['predicted'] = best_model.predict(X[y != y_pred])
print("\nОшибочно классифицированные примеры:")
for idx, row in misclassified.iterrows():
    print(f"Текст: '{row['text']}'")
    print(f"Истинная метка: {'Позитивный' if row['label'] == 1 else 'Негативный'}")
    print(f"Предсказанная метка: {'Позитивный' if row['predicted'] == 1 else 'Негативный'}")
    print("-" * 50)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sentence_transformers import SentenceTransformer
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# Пример данных (категории новостей)
data = pd.DataFrame({
    'text': [
        'Парламент принял новый закон о налогообложении',
        'Сборная выиграла чемпионат мира по футболу',
        'Ученые обнаружили новый вид рыб в Тихом океане',
        'Индекс S&P 500 показал рекордный рост',
        'Депутаты обсуждают поправки в конституцию',
        'Баскетбольная команда выиграла национальный кубок',
        'Исследователи разработали новый метод лечения рака',
        'Центральный банк снизил ключевую ставку',
        'Президент подписал указ о новых экономических мерах',
        'Олимпийская сборная завоевала 10 золотых медалей',
        'Ученые запустили зонд к Марсу',
        'Биржевые индексы упали на фоне новостей из Китая'
    ],
    'category': [
        'политика', 'спорт', 'наука', 'экономика',
        'политика', 'спорт', 'наука', 'экономика',
        'политика', 'спорт', 'наука', 'экономика'
    ]
})

# Загрузка модели Sentence-BERT
sentence_model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

# Получение эмбеддингов предложений
embeddings = sentence_model.encode(data['text'].tolist())

# Кодирование категорий
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(data['category'])

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(embeddings, y, test_size=0.3, random_state=42, stratify=y)

# Создание ансамбля классификаторов
estimators = [
    ('svm', SVC(probability=True, kernel='linear')),
    ('dt', DecisionTreeClassifier()),
    ('lr', LogisticRegression())
]
ensemble = VotingClassifier(estimators=estimators, voting='soft')

# Обучение ансамбля
ensemble.fit(X_train, y_train)

# Оценка модели
y_pred = ensemble.predict(X_test)
print(f"Точность ансамбля: {accuracy_score(y_test, y_pred)}")
print(classification_report(y_test, y_pred, target_names=le.classes_))

# Визуализация эмбеддингов с помощью t-SNE
pca = PCA(n_components=50)
reduced_embeddings = pca.fit_transform(embeddings)

tsne = TSNE(n_components=2, random_state=42)
tsne_embeddings = tsne.fit_transform(reduced_embeddings)

plt.figure(figsize=(10, 8))
for cat_id, cat_name in enumerate(le.classes_):
    idx = y == cat_id
    plt.scatter(tsne_embeddings[idx, 0], tsne_embeddings[idx, 1], label=cat_name)

plt.legend()
plt.title('t-SNE визуализация эмбеддингов новостей')
plt.xlabel('t-SNE 1')
plt.ylabel('t-SNE 2')
plt.show()

# Проверка модели на новых текстах
new_texts = [
    'Правительство обсуждает новые меры поддержки малого бизнеса',
    'Теннисист выиграл турнир Большого шлема',
    'Исследователи выявили новые свойства графена',
    'Инфляция снизилась до рекордных значений'
]

new_embeddings = sentence_model.encode(new_texts)
predictions = ensemble.predict(new_embeddings)
pred_proba = ensemble.predict_proba(new_embeddings)

print("\nПредсказания для новых текстов:")
for i, text in enumerate(new_texts):
    predicted_class = le.classes_[predictions[i]]
    class_probas = {le.classes_[j]: prob for j, prob in enumerate(pred_proba[i])}
    top_classes = sorted(class_probas.items(), key=lambda x: x[1], reverse=True)
    
    print(f"Текст: '{text}'")
    print(f"Предсказанная категория: {predicted_class}")
    print(f"Вероятности: ", end="")
    for cls, prob in top_classes:
        print(f"{cls}: {prob:.4f}, ", end="")
    print("\n" + "-" * 50)

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import matplotlib.pyplot as plt
from tqdm import tqdm
import seaborn as sns
import nltk
from nltk.corpus import stopwords
import random

# Устанавливаем seed для воспроизводимости
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    
set_seed(42)

# Проверка наличия GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Используется устройство: {device}")

# Пример данных (большее количество для эксперта)
# В реальном сценарии используйте реальный датасет
data = pd.DataFrame({
    'text': [
        'Парламент принял новый закон о налогообложении предприятий малого бизнеса',
        'Сборная России выиграла чемпионат мира по футболу в напряженном матче',
        'Ученые обнаружили новый вид глубоководных рыб в Тихом океане',
        'Индекс S&P 500 показал рекордный рост на фоне позитивных новостей',
        'Депутаты обсуждают поправки в конституцию о социальных гарантиях',
        'Баскетбольная команда выиграла национальный кубок в финальном матче',
        'Исследователи разработали новый метод лечения онкологических заболеваний',
        'Центральный банк снизил ключевую ставку до исторического минимума',
        'Президент подписал указ о новых экономических мерах поддержки',
        'Олимпийская сборная завоевала десять золотых медалей на соревнованиях',
        'Ученые запустили исследовательский зонд к поверхности Марса',
        'Биржевые индексы упали на фоне новостей из Китая о замедлении роста',
        'Премьер-министр провел переговоры с лидерами европейских стран',
        'Чемпион мира по боксу защитил свой титул в тяжелом бою',
        'Астрономы наблюдали редкое космическое явление в галактике Андромеды',
        'Нефтяные котировки выросли после заседания ОПЕК о сокращении добычи',
        'Министр иностранных дел выступил с заявлением о ситуации на Ближнем Востоке',
        'Хоккейная команда одержала победу в серии пенальти в финале чемпионата',
        'Генетики расшифровали геном редкого вида насекомых из тропиков',
        'Эксперты прогнозируют рост ВВП страны на 3% в следующем квартале'
    ],
    'category': [
        'политика', 'спорт', 'наука', 'экономика',
        'политика', 'спорт', 'наука', 'экономика',
        'политика', 'спорт', 'наука', 'экономика',
        'политика', 'спорт', 'наука', 'экономика',
        'политика', 'спорт', 'наука', 'экономика'
    ]
})

# Кодирование категорий
le = LabelEncoder()
data['label'] = le.fit_transform(data['category'])
num_classes = len(le.classes_)

# Класс для создания датасета PyTorch
class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        
        # Токенизация текста
        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            return_token_type_ids=True,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'token_type_ids': encoding['token_type_ids'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

# Модель с механизмом внимания поверх BERT
class BertWithAttention(nn.Module):
    def __init__(self, bert_model_name, num_classes, dropout_rate=0.3):
        super(BertWithAttention, self).__init__()
        
        # Загрузка предобученной модели BERT
        self.bert = BertModel.from_pretrained(bert_model_name)
        
        # Размер скрытого состояния BERT
        self.hidden_size = self.bert.config.hidden_size
        
        # Механизм внимания
        self.attention = nn.Sequential(
            nn.Linear(self.hidden_size, 128),
            nn.Tanh(),
            nn.Linear(128, 1),
            nn.Softmax(dim=1)
        )
        
        # Слои классификатора
        self.classifier = nn.Sequential(
            nn.Linear(self.hidden_size, 256),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(256, num_classes)
        )
    
    def forward(self, input_ids, attention_mask, token_type_ids):
        # Получаем выходы из BERT
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        
        # Последние скрытые состояния (batch_size, sequence_length, hidden_size)
        sequence_output = outputs.last_hidden_state
        
        # Применяем механизм внимания к последовательности токенов
        attention_weights = self.attention(sequence_output)
        context_vector = torch.sum(attention_weights * sequence_output, dim=1)
        
        # Классификация на основе взвешенного вектора контекста
        logits = self.classifier(context_vector)
        
        return logits, attention_weights

# Подготовка данных
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    data['text'].values, data['label'].values, 
    test_size=0.2, random_state=42, stratify=data['label']
)

# Создание датасетов
train_dataset = TextClassificationDataset(X_train, y_train, tokenizer)
test_dataset = TextClassificationDataset(X_test, y_test, tokenizer)

# Создание загрузчиков данных
batch_size = 4
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Инициализация модели
model = BertWithAttention('bert-base-multilingual-cased', num_classes)
model = model.to(device)

# Параметры оптимизации
optimizer = AdamW(
    [
        {"params": model.bert.parameters(), "lr": 2e-5},
        {"params": model.classifier.parameters(), "lr": 1e-3},
        {"params": model.attention.parameters(), "lr": 1e-3}
    ],
    weight_decay=0.01
)

# Количество шагов обучения
num_epochs = 5
total_steps = len(train_loader) * num_epochs

# Планировщик скорости обучения с разогревом
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

# Функция обучения
def train_epoch(model, data_loader, optimizer, scheduler, device):
    model.train()
    total_loss = 0
    
    progress_bar = tqdm(data_loader, desc="Training")
    for batch in progress_bar:
        # Извлечение данных из батча
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        labels = batch['label'].to(device)
        
        # Очистка градиентов
        optimizer.zero_grad()
        
        # Прямой проход
        logits, _ = model(input_ids, attention_mask, token_type_ids)
        
        # Функция потерь (кросс-энтропия)
        loss_fn = nn.CrossEntropyLoss()
        loss = loss_fn(logits, labels)
        
        # Обратное распространение ошибки
        loss.backward()
        
        # Предотвращение взрыва градиентов
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        # Обновление весов
        optimizer.step()
        scheduler.step()
        
        total_loss += loss.item()
        progress_bar.set_postfix({"loss": loss.item()})
    
    return total_loss / len(data_loader)

# Функция оценки
def evaluate(model, data_loader, device):
    model.eval()
    predictions = []
    true_labels = []
    attention_maps = []
    
    with torch.no_grad():
        for batch in tqdm(data_loader, desc="Evaluating"):
            # Извлечение данных из батча
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            labels = batch['label'].to(device)
            
            # Прямой проход
            logits, attention_weights = model(input_ids, attention_mask, token_type_ids)
            
            # Сохранение весов внимания
            attention_maps.append(attention_weights.cpu().numpy())
            
            # Получение предсказаний
            _, preds = torch.max(logits, dim=1)
            
            # Сохранение предсказаний и истинных меток
            predictions.extend(preds.cpu().tolist())
            true_labels.extend(labels.cpu().tolist())
    
    return predictions, true_labels, np.vstack(attention_maps)

# Обучение модели
train_losses = []

for epoch in range(num_epochs):
    print(f"Эпоха {epoch+1}/{num_epochs}")
    
    # Обучение на одной эпохе
    train_loss = train_epoch(model, train_loader, optimizer, scheduler, device)
    train_losses.append(train_loss)
    
    print(f"Средняя потеря на обучении: {train_loss:.4f}")
    
    # Оценка на тестовых данных
    predictions, true_labels, _ = evaluate(model, test_loader, device)
    
    # Вывод метрик классификации
    print("\nМетрики на тестовой выборке:")
    print(classification_report(true_labels, predictions, target_names=le.classes_))
    print("-" * 50)

# Визуализация процесса обучения
plt.figure(figsize=(10, 6))
plt.plot(range(1, num_epochs+1), train_losses, 'b-o', label='Потери на обучении')
plt.title('Динамика функции потерь во время обучения')
plt.xlabel('Эпоха')
plt.ylabel('Потеря')
plt.legend()
plt.grid(True)
plt.show()

# Оценка модели на тестовых данных
test_predictions, test_true_labels, attention_maps = evaluate(model, test_loader, device)

# Визуализация внимания на тестовых примерах
def visualize_attention(text, attention_weights, tokenizer):
    # Токенизация текста
    tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(text)))
    
    # Получение весов внимания для токенов
    attention = attention_weights.reshape(-1)[:len(tokens)]
    
    # Создание визуализации
    plt.figure(figsize=(10, 6))
    sns.barplot(x=attention, y=tokens)
    plt.title('Веса внимания для каждого токена')
    plt.xlabel('Вес внимания')
    plt.tight_layout()
    plt.show()

# Выбор случайного примера для визуализации
sample_idx = np.random.randint(len(X_test))
sample_text = X_test[sample_idx]
sample_label = y_test[sample_idx]
sample_pred = test_predictions[sample_idx]
sample_attention = attention_maps[sample_idx // batch_size][sample_idx % batch_size]

print(f"Текст: '{sample_text}'")
print(f"Истинная категория: {le.classes_[sample_label]}")
print(f"Предсказанная категория: {le.classes_[sample_pred]}")
visualize_attention(sample_text, sample_attention, tokenizer)

# Интерпретация результатов с помощью SHAP
import shap

# Создаем объяснительную модель
explainer = shap.Explainer(lambda x: model(
    torch.tensor(tokenizer(x, padding=True, truncation=True, return_tensors="pt")["input_ids"]).to(device),
    torch.tensor(tokenizer(x, padding=True, truncation=True, return_tensors="pt")["attention_mask"]).to(device),
    torch.tensor(tokenizer(x, padding=True, truncation=True, return_tensors="pt")["token_type_ids"]).to(device)
)[0].detach().cpu().numpy(), tokenizer)

# Выбираем несколько примеров для объяснения
examples = X_test[:5]
shap_values = explainer(examples)

# Визуализация SHAP значений
shap.plots.text(shap_values)